## Nemo Assembly Calculus

Github Repo [link](https://github.com/dmitropolsky/assemblies)

In [80]:
import numpy as np
import heapq
import matplotlib.pyplot as plt
import collections
from scipy.stats import binom
from scipy.stats import truncnorm
from scipy.stats import norm
import math
import types
import sys
sys.path.append('/Users/samart/Desktop/Neuro/SNNs/assemblies')
# Configurable assembly model for simulations
# Author Daniel Mitropolsky, 2018

EMPTY_MAPPING = types.MappingProxyType({})

# Iterating Core Components

In [81]:
import brain

In [69]:
N = 100
k = 10

p = 0.1
beta = 0.05

### Brain Areas

- Core Object of NEMO is a named area, N neurons with $G_{n,p}$ connectivity (independent probability p of directed pairwise connection)
- Explicit makes us actually simulate everything while False makes us lazy evaluate



In [75]:
area = brain.Area(name = 'A', n=N, k=k, beta=beta, explicit=True)

Can access all the core components of area; mostly a container class without much "action" to it

In [76]:
area.name, area.n, area.k, area.beta, area.explicit

('A', 100, 10, 0.05, True)

Can selectively have different update weights for different neighboring areas or stimuli

In [74]:
area.beta_by_stimulus, area.beta_by_area

({}, {})

- Each area stores the k top winners that fired in the most recent timestep in area.winners
- Stores the history of all previous fired winners in area.saved_winners
- Analogous for w and saved_w; these

In [77]:
area.w, area.winners, area.saved_winners, area.saved_winners

(0, [], [], [])

### Constructing Brain Area Graph

The critical object is the Brain, which is basically a Graph of connected areas and stimuli (which are distinct). 
- brain tracks all the associated areas and stimuli in dictionaries (keys are names of each)
- tracks edges/ connections in connectomes dictionaries, basically adjacency list format
    - connections are made between any two neurons in areas with brain_probability p (set to same p as above but in principle could be different)
-  

In [110]:
n = 10
k = 2
p = 0.5
beta = 0.05
t = 10

In [111]:
B = brain.Brain(p=p, save_size=True, save_winners=True, seed=10)
B

In [112]:
B.add_explicit_area('A', n, k, beta)
B.add_explicit_area('B', n+1, k, beta)
B.add_explicit_area('C', n+2, k, beta)

B.add_stimulus(stimulus_name="STIM_1", size=5)
B.add_stimulus(stimulus_name="STIM_2", size=5)

In [113]:
B.area_by_name

{'A': <brain.Area at 0x112148150>,
 'B': <brain.Area at 0x11214a3d0>,
 'C': <brain.Area at 0x11217ad90>}

In [114]:
B.stimulus_size_by_name

{'STIM_1': 5, 'STIM_2': 5}

Connectomes are added for each area; these (matrices are created as binomial matrices with p probability of being connected

- two areas with $n_a$ and $n_b$ are connected with $(n_a, n_b)$ adjacency matrix. $p(A_{ij}=1)=p$
- A stimulus of size $s$ is connected to a area of size $n$ with a $n$-vector, each element being the number of stimulus nodes connected (how much stimulus activates each neuron in area). This is calculated again by binomial distribution, but this time by counting successes in $s$ trials with probability p. Essentially, shortcut the matrix by not physically representing stimulus neurons. 

In [115]:
B.connectomes['A']['B'].shape, B.connectomes['A']['B']

((10, 11),
 array([[0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
        [1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0.],
        [0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1.],
        [1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0.],
        [0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1.],
        [0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0.]], dtype=float32))

In [118]:
# binomial(n=stim_size, p=p, size = n_area)
B.connectomes_by_stimulus['STIM_1']

{'A': array([1., 3., 3., 3., 2., 5., 2., 1., 3., 0.], dtype=float32),
 'B': array([5., 1., 3., 3., 4., 3., 4., 2., 3., 2., 2.], dtype=float32),
 'C': array([4., 2., 4., 3., 4., 5., 3., 4., 1., 4., 1., 4.], dtype=float32)}

In [122]:
B.area_by_name['A'].winners # still no winners!

[]

### Projection and Activation

Project takes in 2 primary arguments: 
- areas_by_stim is a dictionary where keys are stimuli name and values are the areas want to project that stimulus onto. $\{$ "stim1":$[A]$, "stim2":$[C,A]\}$
- dst_areas_by_src areas is a dictionary where keys and values are both 


In [129]:
B.activate('C', index=3) # zap neurons [index*k : index*(k+1)] to enforce firing

In [127]:
B.area_by_name['C'].winners

[6, 7]

### Simple Experiment

Filipp Code Explanation :

- When you initialize a stimulus, it is [initialized to nothing](https://github.com/dmitropolsky/assemblies/blob/master/brain.py#L153)
- When stimuli and/or areas or added, automatically have some probability of being [connected](https://github.com/dmitropolsky/assemblies/blob/master/brain.py#L170) to each other
- When you project, it's going to call the [project_into method](https://github.com/dmitropolsky/assemblies/blob/master/brain.py#L300) with your stimulus and the area
- Then it stimulates areas to [find k winners here](https://github.com/dmitropolsky/assemblies/blob/master/brain.py#L398-L417)
    - The "k" that you previously defined for your stimulus affects the amount of stimulation. That's why it makes sense to set k=k (where the other k is the k in top-k). It uses this to randomly fire that number of neurons.
- Finally the resulting winners are now [associated with the stimulus](https://github.com/dmitropolsky/assemblies/blob/master/brain.py#L480-L516)
- The next time you project, the previous winners that are associated with the stimulus [will be taken into account](https://github.com/dmitropolsky/assemblies/blob/master/brain.py#L340-L351)

Increasing the likelihood of them activating again. So essentially as you keep on firing the stimulus the assemblies will start to converge.

In [29]:
n = 10
k = 2
p = 0.01
beta = 0.05
t = 10

In [30]:
stimulus = "STIM"
basic_brain = brain.Brain(p)
basic_brain.add_stimulus(stimulus_name=stimulus,size=k)
basic_brain.add_explicit_area("A",n,k,beta)


In [42]:
type(basic_brain._rng)

numpy.random._generator.Generator

In [52]:
stim_size = 10

basic_brain._rng.binomial#(stim_size, p=0.4, size=4)

<function Generator.binomial>

In [38]:
basic_brain.area_by_name['A'].n

10

In [ ]:
this_stimulus_connectomes[area_name] = self._rng.binomial(
    size, self.p,
    size=self.area_by_name[area_name].n).astype(np.float32)

In [33]:
basic_brain.connectomes_by_stimulus['STIM']

{'A': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)}

In [21]:
basic_brain.area_by_name

{'A': <brain.Area at 0x12a396f90>}

In [26]:
basic_brain.project({stimulus:["A"]},{})

In [ ]:


for i in range(t-1):
   basic_brain.project({stimulus:["A"]},{"A":["A"]})

area = basic_brain.area_by_name["A"]
#self.assertEqual(w[-2], w[-1])


"""
w: Number of neurons that has ever fired in this area.
saved_w: List of per-round size-of-support.
winners: List of winners, as set by previous action.
saved_winners: List of lists of all winners, per-round.
"""
print(f"Number of neurons fired: {area.w}")
print(f"Saved_w: {area.saved_w}")
print(f"Winners: {area.winners}")
print(f"Saved_Winners: {area.saved_winners}\n")